In [1]:
import torch
import numpy as np
import sqlite3
import timm
import tabulate
import pyperclip
import db_utils as dbu
import torch.nn.functional as F

%matplotlib inline 
%load_ext autoreload
%autoreload 2

device = 'cpu'# 'cuda' if torch.cuda.is_available() else 'cpu'

model_names = ['vit_base_patch16_224', 
               'vit_base_patch16_224_miil', 
               'vit_base_patch32_224', 
               'vit_large_patch16_224']
pretty_model_names = {
    'vit_base_patch16_224': 'ViT-B/16', 
    'vit_base_patch16_224_miil': 'ViT-B/16-MIIL', 
    'vit_base_patch32_224': 'ViT-B/32', 
    'vit_large_patch16_224': 'ViT-L/16'
}
models = {
    model_name: timm.create_model(model_name, pretrained=True).eval().to(device)
    for model_name in model_names
}

## TODO
Make sure the `imagenet_val_evaluator.py` script has been executed. Fill in the path to the bias activations below here.

In [2]:
bias_pl_resid_path = 'A:\\CVData\\ImageNet\\val_top_bias_pl_resid'
bias_pl_vec_noise_path = 'A:\\CVData\\ImageNet\\val_top_bias_pl_vec_noise'
bias_pl_all_path = 'A:\\CVData\\ImageNet\\val_top_bias_pl_all'
db_path = 'imgnet_val_prelim.db'

## Naïve Bias Addition to Value Vectors
Here the bias is added to every extracted value vector. Then these vectors are projected with the MLP head and the most predictive vector is determined by identifying the vector with the highest class score. Based on the concept that for blocks where classes have their most predictive value vectors, the following holds true:

$$
\begin{aligned}
	\bm{t}_{\text{cls}}^{(b^{v_c^*},\text{MLP})} & \approx \bm{b}_V^{(b^{v_c^*},\text{MLP})} + \bm{v}_c^*.
\end{aligned}
$$

In [3]:
from src.utils.extraction import extract_value_vectors, extract_value_biases
from src.utils.model import embedding_projection
from src.analyzers.vector_analyzer import most_predictive_ind_for_classes

values = {
    model_name: extract_value_vectors(models[model_name], device, stack=True)
    for model_name in model_names
}
values_with_bias = {
    model_name: extract_value_vectors(models[model_name], device, True, 1.0)
    for model_name in model_names
}
projected_values = {
    model_name: embedding_projection(models[model_name], values[model_name], device)
    for model_name in model_names
}
projected_values_with_bias = {
    model_name: embedding_projection(models[model_name], values_with_bias[model_name], device)
    for model_name in model_names
}
most_predictive = {
    model_name: most_predictive_ind_for_classes(projected_values[model_name], device)
    for model_name in model_names
}
most_predictive_with_bias = {
    model_name: most_predictive_ind_for_classes(projected_values_with_bias[model_name], device)
    for model_name in model_names
}

avg_cls_score = {
    model_name: projected_values[model_name][most_predictive[model_name][0,:], 
                                             most_predictive[model_name][1,:], 
                                             most_predictive[model_name][2,:]].mean().item()
    for model_name in model_names
}
avg_cls_score_with_bias = {
    model_name: projected_values_with_bias[model_name][most_predictive[model_name][0,:], 
                                                       most_predictive[model_name][1,:], 
                                                       most_predictive[model_name][2,:]].mean().item()
    for model_name in model_names
}
avg_bias_predictive_cls_score_with_bias = {
    model_name: projected_values_with_bias[model_name][most_predictive_with_bias[model_name][0,:], 
                                                       most_predictive_with_bias[model_name][1,:], 
                                                       most_predictive_with_bias[model_name][2,:]].mean().item()
    for model_name in model_names
}

In [4]:
avg_cls_score_val = [row[1] for row in dbu.sort_like(
    model_names, 
    dbu.select(db_path, 'select pred_model, avg(top_0_score) from predictions group by pred_model'))]

predictive_vector_weights = dbu.select(
    db_path, 'select pred_model, num_idx, avg(top_0_cls_token_act) from vec_activations '
    'group by pred_model, num_idx order by pred_model, num_idx')
predictive_vector_weights = dbu.group_by_tensor(predictive_vector_weights, 0, 2, True)

cls_scores_around_most_pred = dbu.select(
    db_path, 'select pred_model, num_idx, avg(before_most_predictive_pred), '
    'avg(after_most_predictive_pred), avg(cls_score_bias_pl_all_pred) from vec_activations group by '
    'pred_model, num_idx order by pred_model, num_idx'
)
score_before_most_pred = dbu.group_by_tensor(cls_scores_around_most_pred, 0, 2, True)
score_after_most_pred = dbu.group_by_tensor(cls_scores_around_most_pred, 0, 3, True)
score_bias_pl_all_pred = dbu.group_by_tensor(cls_scores_around_most_pred, 0, 4, True)

avg_cls_score_with_bias_weighted = {}
avg_percentage_score_with_bias_weighted = {}
value_biases = {
        model_name: extract_value_biases(models[model_name], device, True)
        for model_name in model_names
}

for model_name in model_names:
    most_pred_values = values[model_name][most_predictive[model_name][0,:], 
                                          most_predictive[model_name][1,:]]
    most_pred_values *= F.gelu(predictive_vector_weights[model_name].to(device)[:, None])
    most_pred_biases = value_biases[model_name][most_predictive[model_name][0,:]]
    cls_score_with_bias_weighted = torch.diagonal(embedding_projection(
            models[model_name], most_pred_values + most_pred_biases
    ))
    avg_percentage_score_with_bias_weighted[model_name] = (cls_score_with_bias_weighted / 
        (score_after_most_pred[model_name] - score_before_most_pred[model_name])).mean().item()
    avg_cls_score_with_bias_weighted[model_name] = cls_score_with_bias_weighted.mean().item()

In [7]:
from IPython.display import display_markdown

tbl_headers = ['Average of'] + [pretty_model_names[model_name] for model_name in model_names]
bias_comparison = [['$(\\bm{v}_c^* E)_c$'] + [avg_cls_score[model_name] for model_name in model_names], 
                     ['$((\\bm{b}_V^{(b^{v_c^*},\\text{MLP})} + \\bm{v}_c^*) E)_c$'] + 
                     [avg_cls_score_with_bias[model_name] for model_name in model_names],
                     ['$(\\bm{t}_{\\text{cls}}^{\\text{pred}} E)_c$'] + avg_cls_score_val,
                     ['$((\\bm{b}_V^{(b^{v_c^*},\\text{MLP})} + \\text{GELU}\\left(w^{v^*_c}_{\\text'  
                      '{cls}}\\right)\\bm{v}_c^*) \\cdot E)_c$'] + 
                     [avg_cls_score_with_bias_weighted[model_name] for model_name in model_names],
                     ['percentage'] + 
                     [avg_percentage_score_with_bias_weighted[model_name] for model_name in model_names]]

display_markdown(
    tabulate.tabulate(bias_comparison, tbl_headers, floatfmt='.3f', tablefmt='github'), raw=True)

| Average of                                                                                                   |   ViT-B/16 |   ViT-B/16-MIIL |   ViT-B/32 |   ViT-L/16 |
|--------------------------------------------------------------------------------------------------------------|------------|-----------------|------------|------------|
| $(\bm{v}_c^* E)_c$                                                                                           |      9.150 |          10.477 |     10.680 |      9.837 |
| $((\bm{b}_V^{(b^{v_c^*},\text{MLP})} + \bm{v}_c^*) E)_c$                                                     |      0.361 |           4.180 |      1.054 |      1.389 |
| $(\bm{t}_{\text{cls}}^{\text{pred}} E)_c$                                                                    |     13.676 |          10.825 |     13.916 |     13.880 |
| $((\bm{b}_V^{(b^{v_c^*},\text{MLP})} + \text{GELU}\left(w^{v^*_c}_{\text{cls}}\right)\bm{v}_c^*) \cdot E)_c$ |      1.068 |           2.759 |      2.346 |      2.047 |
| percentage                                                                                                   |      0.713 |           0.935 |      0.786 |      0.675 |

In [5]:
dbu.get_column_names(db_path, 'vec_activations')

['path',
 'pred_model',
 'imagenet_id',
 'num_idx',
 'name',
 'top_0_cls_token_act',
 'top_1_cls_token_act',
 'top_2_cls_token_act',
 'top_3_cls_token_act',
 'top_4_cls_token_act',
 'top_5_cls_token_act',
 'top_6_cls_token_act',
 'top_7_cls_token_act',
 'top_8_cls_token_act',
 'top_9_cls_token_act',
 'top_0_img_token_avg_act',
 'top_1_img_token_avg_act',
 'top_2_img_token_avg_act',
 'top_3_img_token_avg_act',
 'top_4_img_token_avg_act',
 'top_5_img_token_avg_act',
 'top_6_img_token_avg_act',
 'top_7_img_token_avg_act',
 'top_8_img_token_avg_act',
 'top_9_img_token_avg_act',
 'top_0_block_ind',
 'top_1_block_ind',
 'top_2_block_ind',
 'top_3_block_ind',
 'top_4_block_ind',
 'top_5_block_ind',
 'top_6_block_ind',
 'top_7_block_ind',
 'top_8_block_ind',
 'top_9_block_ind',
 'top_0_vec_ind',
 'top_1_vec_ind',
 'top_2_vec_ind',
 'top_3_vec_ind',
 'top_4_vec_ind',
 'top_5_vec_ind',
 'top_6_vec_ind',
 'top_7_vec_ind',
 'top_8_vec_ind',
 'top_9_vec_ind',
 'top_0_max_cls_token_act',
 'top_1_max

In [11]:
predictive_vector_weights = dbu.select(
    db_path, 'select pred_model, num_idx, avg(top_0_cls_token_act) from vec_activations '
    'group by pred_model, num_idx order by pred_model, num_idx')
predictive_vector_weights = dbu.group_by_tensor(predictive_vector_weights, 0, 2)

In [25]:
dbu.select(db_path, 'select pred_model, avg(before_most_predictive_pred), avg(after_most_predictive_pred),'
           ' avg(cls_score_all_pred) from vec_activations group by pred_model')

[('vit_base_patch16_224',
  1.1954224396071833,
  4.127169065102373,
  2.9059458256638515),
 ('vit_base_patch16_224_miil',
  4.737318438119581,
  7.441784518307187,
  6.82038210035231),
 ('vit_base_patch32_224',
  1.1713416013254738,
  3.0311404356388447,
  2.4313932356904635),
 ('vit_large_patch16_224',
  4.11039840854872,
  6.764502003012034,
  5.352589148541773)]

In [13]:
models[model_names[0]]

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p=0.0, inplace=F